In [153]:
import folium
import sys
import pandas as pd
import geopandas as gpd
sys.path.insert(0, 'python_scripts')

In [200]:
import sql_functions
reload(sql_functions)
month = '1996_06'

In [201]:
sh = sql_functions.statehouse()

In [233]:
j=sh.iloc[:,[2,10]]
df = sh[['regionName',month]]
print df
print j
j =j[j[month] > 0]
#j.filter(like='arg in '+month+' > 0')

    regionName  1996_06
0        21234    98215
1        20906   144925
2        21740    82825
3        21117   156748
4        20878   212161
5        21215    58725
6        21222    74525
7        21122   125425
8        20874   147660
9        21224    53400
10       21061   111903
11       20852   185435
12       20904   166800
13       20910   188100
14       21218    42325
15       21206    71265
16       21207    85213
17       21228   120600
18       20744   153375
19       20850   235890
20       21229    72765
21       20774   152450
22       21044   172947
23       21221    84100
24       21401   148690
25       21502    46700
26       20902   145150
27       21043   195725
28       20854   302750
29       21921   117790
..         ...      ...
244      21710        0
245      21782        0
246      21919        0
247      21830        0
248      21658        0
249      21660        0
250      21915        0
251      21661        0
252      21140        0
253      21561  

In [208]:
json_path = 'topo_files/MD_zip.json'
gdf = gpd.read_file(json_path)

In [209]:
gdf.head()

,ALAND10,AWATER10,CLASSFP10,FUNCSTAT10,GEOID10,INTPTLAT10,INTPTLON10,MTFCC10,PARTFLG10,STATEFP10,ZCTA5CE10,geometry
0,9382408,85941,B5,S,2421529,+39.7039417,-078.7671714,G6350,N,24,21529,"POLYGON ((-78.77455500000001 39.72298, -78.774..."
1,41454184,0,B5,S,2421144,+39.1209124,-076.6772681,G6350,N,24,21144,"POLYGON ((-76.687494 39.149036, -76.6875090000..."
2,33861012,186378,B5,S,2421108,+39.0912665,-076.6210983,G6350,N,24,21108,"(POLYGON ((-76.636743 39.123324, -76.63687 39...."
3,32403463,2722648,B5,S,2421060,+39.1697908,-076.5808725,G6350,N,24,21060,"POLYGON ((-76.584621 39.134111, -76.5849330000..."
4,109677132,3354891,B5,S,2421120,+39.6462041,-076.6748540,G6350,N,24,21120,"POLYGON ((-76.63052500000001 39.652713, -76.62..."


In [210]:
merged = gdf.merge(j, left_on='ZCTA5CE10', right_on='regionName')

In [211]:
merged.iloc[:,[11,12]].head()

,geometry,regionName
0,"POLYGON ((-76.687494 39.149036, -76.6875090000...",21144
1,"POLYGON ((-76.584621 39.134111, -76.5849330000...",21060
2,"POLYGON ((-76.63052500000001 39.652713, -76.62...",21120
3,"POLYGON ((-76.516238 39.356892, -76.5174240000...",21236
4,"POLYGON ((-76.75212399999999 39.306124, -76.75...",21228


In [212]:
spatial_gdf = gpd.GeoDataFrame(merged.iloc[:, [11, 12]])

In [213]:
data_f = merged.iloc[:, [11,12,13]]

In [214]:
data_f.head()

,geometry,regionName,1996_06
0,"POLYGON ((-76.687494 39.149036, -76.6875090000...",21144,150685
1,"POLYGON ((-76.584621 39.134111, -76.5849330000...",21060,112600
2,"POLYGON ((-76.63052500000001 39.652713, -76.62...",21120,228000
3,"POLYGON ((-76.516238 39.356892, -76.5174240000...",21236,106820
4,"POLYGON ((-76.75212399999999 39.306124, -76.75...",21228,120600


In [235]:
geo_str = spatial_gdf.to_json()

In [227]:
map = folium.Map(location=[38.9784, -76.4922], zoom_start=9)
folium.TileLayer('cartodbpositron').add_to(map)

In [228]:
min_val = j['1996_06'].min()
q1 = j['1996_06'].quantile( .25)
q2 = j['1996_06'].quantile( .5)
q3 = j['1996_06'].quantile( .75)

#help(map.choropleth)
min_val,q1,q2,q3

(28500, 111459.0, 134890.0, 157875.0)

In [229]:
map.choropleth(geo_str=geo_str,
              data=data_f,
              columns=['regionName', '1996_06'],
              fill_color='YlOrRd',
              key_on='feature.properties.regionName',
              threshold_scale=[min_val, q1, q2, q3],
              legend_name='Median of Sold Houses',
              reset=True)

In [230]:
map.save('test.html')

In [231]:
folium.__version__

'0.2.1'